# Data_Cleaning_Challenge - Kristofer Schobert

In [240]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv('WELLCOME_APCspend2013_forThinkful_UTF8.csv')

In [241]:
df.head(7)

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88
5,PMC3579457,ACS,Journal of Medicinal Chemistry,Comparative Structural and Functional Studies ...,£2392.20
6,PMC3709265,ACS,Journal of Proteome Research,Mapping Proteolytic Processing in the Secretom...,£2367.95


In [242]:
#dropping unnecessary columns
df = df.drop(columns=['PMID/PMCID', 'Article title'])
df.head()

,Publisher,Journal title,COST (£) charged to Wellcome (inc VAT when charged)
0,CUP,Psychological Medicine,£0.00
1,ACS,Biomacromolecules,£2381.04
2,ACS,J Med Chem,£642.56
3,ACS,J Med Chem,£669.64
4,ACS,J Org Chem,£685.88


In [243]:
#renaming columns
df = df.rename(columns={'Journal title':'Journal', 'COST (£) charged to Wellcome (inc VAT when charged)': 'Cost'})
df.head()

,Publisher,Journal,Cost
0,CUP,Psychological Medicine,£0.00
1,ACS,Biomacromolecules,£2381.04
2,ACS,J Med Chem,£642.56
3,ACS,J Med Chem,£669.64
4,ACS,J Org Chem,£685.88


In [244]:
#view the set of publishers
set(df['Publisher'])

{'ACS',
 'ACS (Amercian Chemical Society) Publications',
 'ACS Publications',
 'AGA Institute',
 'AMBSB',
 'AMERICAN CHEMICAL SOCIETY',
 'ASBMB',
 'ASBMB Cadmus',
 'ASBMB/Cadmus',
 'ASBMB/Cenveo Publisher Services',
 'ASBMC /CENVEO',
 'ASM',
 'ASM (American Society for Microbiology)',
 'American Association of Immunologists',
 'American Chemical Society',
 'American Chemical Society Publications',
 'American College of Chest Physicians',
 'American Physiological Society',
 'American Psychiatric Association',
 'American Psychiatric Publishing',
 'American Psychological Association',
 'American Psychological Association ',
 'American Public Health Association',
 'American Soc for Biochemistry and Molecular Biology',
 'American Society for Biochemistry and Molecular Biolgy',
 'American Society for Biochemistry and Molecular Biology',
 'American Society for Biochemistry and Molecular Biology ',
 'American Society for Investigative Pathology',
 'American Society for Microbiology',
 'America

In [245]:
#cleaning the data
#creating a loop that will pass in each value for a given key and (with some help from regular expressions) 
#turn all differently formatted mentions of a publisher into the key.

regex_desired_dict = {'American Chemical Society':['ACS',
                                                   'AMERICAN CHEMICAL SOCIETY',
                                                   'American Chemical Society'],
                      'American Society for Microbiology':['American Society for Microbiology',
                                                           'ASM'],
                      'American Society for Biochemistry and Molecular Biology':['ASBMB',
                                                                                 'American Society for Biochemistry and Molecular',
                                                                                 'American Soc for Biochemistry and Molecular'],
                      'American Society of Hematology':['American Society of H[ea]*matology'],
                      'American Psychological Association':['American Psychological'],
                      'BioMed Central':['BMC','Bio[Mm]ed [Cc]entral'],
                      'BMJ':['BMJ'],
                      'Bentham Science Publishers':['Bentha[mn] Science Publishers'],
                      'Cadmus Journal Services':['[Cc][Aa][DdMm]+[Uu][Ss]'],
                      'Cambridge University Press':['Cambridge U','CUP'],
                      'Cold Spring Harbor':['Cold Spring Ha'],
                      'Company of Biologists':['Company of Biologi','THE COMPANY OF BIOLOGISTS'],
                      'Dartmouth Journal Services':['Dar[t]*mouth'],
                      'Elsevier':['^E[Ll][Ss][Ee][Vv]'],
                      'Federation of American Societies for Experimental Biology':['FASEB','^Federation of'],
                      'Frontiers':['^Front'],
                      'Future Medicine':['^Future M'],
                      'Hindawi':['Hindawi'],
                      'Impact Journals':['Impact Journals'],
                      'Informa Healthcare':['Informa Healthcare'],
                      'International Union Against Tuberculosis and Lung Disease':['^International Union Against'],
                      'International Union of Crystallography':['International Union of Crystallography'],
                      'Wiley':['W[il]+ey', 'WILEY'],
                      'Journal of Visualized Experiments':['Journal of Visualized Experiments', 'JoVE', 'JOVE'],
                      'Karger':['[Kk][Aa][Rr][Gg][Ee][Rr]'],
                      'Landes Bioscience':['Landes Bioscience'],
                      'Mary Ann Liebert':['Mary Ann Liebert','MARY ANN LIEBERT'],
                      'MIT Press':['MIT'],
                      'Nature Publishing Group':['NATURE PUBLISHING GROUP LTD','NPG'],
                      'National Academy of Sciences':['National Academy of Sciences'],
                      'Nature':['^Nature'],
                      'Oxford University Press':['OUP','^OXFORD','^Oxford'],
                      'Public Library of Science':['^P[Ll][Oo][Ss]'],
                      'PNAS':['PNAS'],
                      'Portland Press':['^Portland','^PORTLAND'],
                      'PubMed':['PubMed'],
                      'Royal Society of Chemistry':['^RSC','^Royal Society'],
                      'Society for Neurosciences':['Society for Neuroscience','SOCIETY OF NEURO SCIENCES'],
                      'Sage Publications':['^Sage','^SAGE'],
                      'Springer':['^Springer', '^SPRINGER'],
                      'Society of General Microbiology':['^Society of Gen','^Society for Gen'],
                      'Society of Leukocyte Biology':['Society of Leukocyte Biology','Society for Leukocyte Biology'],
                      'Society of Neuroscience':['Society of Neuroscience','Society for Neurosciences'],
                      'Taylor & Francis':['T&F','Taylor & Francis','Taylor and Francis'],
                      'The Endocrine Socety':['THE ENDOCRINE SOCIETY','Society for Endocrinology','Endocrine','Endrocrine'],
                      'The Sheridan Press':['The Sheridan Press'],
                      'The Company of Biologists':['^The company of Biol'],
                      'Wolters Kluwer':['^Wolters']
                     }


for key, values in regex_desired_dict.items():
    for value in values:
        df.Publisher = np.where(df.Publisher.str.contains(value, regex=True), 
                           key, df.Publisher)           
set(df.Publisher)         
#showing our cleaned set of publishers



{'AGA Institute',
 'AMBSB',
 'American Association of Immunologists',
 'American Chemical Society',
 'American College of Chest Physicians',
 'American Physiological Society',
 'American Psychiatric Association',
 'American Psychiatric Publishing',
 'American Psychological Association',
 'American Public Health Association',
 'American Society for Biochemistry and Molecular Biology',
 'American Society for Investigative Pathology',
 'American Society for Microbiology',
 'American Society for Nutrition',
 'American Society of Hematology',
 'American Society of Human Genetics (Elsevier)',
 'American Society of Microbiology',
 'American Speech-Language-Hearing Association',
 'Association for Research in Vision & Ophthalmology',
 'BMJ',
 'Bentham Science Publishers',
 'Berhahn Books',
 'BioMed Central',
 'BioScientifica',
 'Biochem Journal',
 'Biophysical Society',
 'Bioscientifica',
 'Brill',
 'British Medical Journal',
 'Byophysical Society',
 'COACTION',
 'CSHLP',
 'Cadmus Journal Servi

## Here are the top 5 publishers and the number of articles for each:

In [246]:
df.groupby('Publisher').count().sort_values(by='Journal', ascending=False)['Journal'].head(5)

Publisher
Elsevier                     408
Public Library of Science    307
Wiley                        270
Oxford University Press      167
BioMed Central                96
Name: Journal, dtype: int64

In [247]:
#Let's make sure each cost is as expected, prefixed with '£'
for cost in df.Cost:
    if cost[0] != '£':
        print(cost)

1674$
1375.8$
2010$
1204.38$
1254.6$
1476$
671.04$
1440$
1460.3$
1476.47$
1570.87$
1600.25$
1600.25$


In [248]:
#The above costs are given in what I will assume are dollars. 
#They are the only elements of the cost columns that are not prefixed with a '£'
#I will convert them to dollars using the conversion factor of December 31, 2012
#then update the cost column with these floats.
converted_pounds = []
for cost in df.Cost.str[:]:
    if cost[0] != '£':
        converted_pounds.append(float(cost.replace('$',''))/1.6259)
    else:
        converted_pounds.append(float(cost.replace('£','')))
df.Cost = converted_pounds        

In [249]:
top_5_publishers = df.groupby('Publisher').count().sort_values(by='Journal', ascending=False)['Journal'].head(5).index
top_5_publishers





Index(['Elsevier', 'Public Library of Science', 'Wiley',
       'Oxford University Press', 'BioMed Central'],
      dtype='object', name='Publisher')

## Stats for the top 5 

In [250]:
#'Elsevier'
df.Cost[df.Publisher == top_5_publishers[0]].describe()

count       408.000000
mean      26885.735833
std      154440.193220
min         348.330000
25%        2168.965000
50%        2354.785000
75%        2476.527500
max      999999.000000
Name: Cost, dtype: float64

In [251]:
#"Public Library of Science'
df.Cost[df.Publisher == top_5_publishers[1]].describe()

count       307.000000
mean      50551.276249
std      215820.204127
min         122.310000
25%         877.090000
50%        1023.410000
75%        1446.805000
max      999999.000000
Name: Cost, dtype: float64

In [252]:
#'Wiley'
df.Cost[df.Publisher == top_5_publishers[2]].describe()

count       270.000000
mean      17532.888444
std      121308.114848
min         612.590000
25%        1835.307500
50%        2009.945000
75%        2310.950000
max      999999.000000
Name: Cost, dtype: float64

In [253]:
#'Oxford University Press'
df.Cost[df.Publisher == top_5_publishers[3]].describe()

count       167.000000
mean      19775.350240
std      132975.378101
min         378.780000
25%        1700.000000
50%        2040.000000
75%        2047.750000
max      999999.000000
Name: Cost, dtype: float64

In [256]:
#'BioMed Central'
df.Cost[df.Publisher == top_5_publishers[4]].describe()

count        96.000000
mean      22106.328180
std      143389.501399
min           9.930000
25%        1178.850000
50%        1312.050000
75%        1536.000000
max      999999.000000
Name: Cost, dtype: float64

In [257]:
# These all have a max of 1 million - 1 English Pounds.....

#How many entries are there of this??


df[df.Cost == 999999.000000]

,Publisher,Journal,Cost
149,American Society for Biochemistry and Molecula...,Journal of Biological Chemistry,999999.0
227,BioMed Central,BMC Genomics.,999999.0
277,BioMed Central,Trials,999999.0
358,Cambridge University Press,Expert Reviews in Molecular Medicine,999999.0
404,Company of Biologists,Journal of Cell Science,999999.0
410,Cambridge University Press,Journal of the International African Institute,999999.0
491,Elsevier,Cell,999999.0
560,Elsevier,Experimental Cell Research,999999.0
630,Elsevier,Journal of paediatric Urology,999999.0
660,Elsevier,Molecluar & Cellular Endocrinology,999999.0


Well maybe the upperlimit of the cost data one could enter was 999999.00, but wouldn't they have added 99 cents? I'm not sure what to make of this messy, messy data... 